In [1]:
import pandas as pd

In [2]:
resfile = "gpt-3.5-turbo_system_results_singleprompt.csv"
#resfile = "gpt-3.5-turbo_system_results.csv"
results = pd.read_csv(resfile)
results["pairid"] = results["pairid"].str[4:]
results["pairid"] = results["pairid"].astype(int)
if "singleprompt" in resfile:
    results["CorrectACauseB"] = results["CorrectCause"]
    results["CorrectBCauseA"] = results["CorrectCause"]
results["CorrectACauseB"] = results["CorrectACauseB"].astype(float)
results["CorrectBCauseA"] = results["CorrectBCauseA"].astype(float)
results

,CorrectACauseB,CorrectBCauseA,pairid,varA,varB,dataset,groundtruth,CorrectCause
0,1.0,1.0,1,Altitude,Temperature,DWD,->,1
1,1.0,1.0,2,Altitude,Precipitation,DWD,->,1
2,0.0,0.0,3,Longitude,Temperature,DWD,->,0
3,1.0,1.0,4,Altitude,Sunshine hours,DWD,->,1
4,1.0,1.0,5,Age,Length,Abalone,->,1
...,...,...,...,...,...,...,...,...
103,1.0,1.0,104,time for passing 1. segment,time for passing 2. segment,D. Janzing,->,1
104,1.0,1.0,105,pixel vector of a patch,total brightness at the screen,D. Janzing,->,1
105,1.0,1.0,106,time required for one round,voltage,D. Janzing,<-,1
106,1.0,1.0,107,strength of contrast,answer correct or not,"Schuett, edited by D. Janzing",->,1


In [3]:
# Computing accuracy
# The first column denotes whether chatGPT answered correctly for the question, "Does A cause B?"
# The second column denotes whether chatGPT answered correctly for the question, "Does B cause A?".
# if the result is 0,1 or 1,0, then we can take a random guess to decide the direction of causation. 
# Therefore, to compute accuracy, we can take the mean of "Does A cause B" and "Does B cause A" questions. 
results["accuracy"] = (results["CorrectACauseB"] + results["CorrectBCauseA"])/2

In [4]:
weights = pd.read_table("pairmeta.txt", sep=" ", header=None, names=["pairid", "v1", "v2", "v3", "v4", "weight"])
weights

,pairid,v1,v2,v3,v4,weight
0,1,1,1,2,2,0.166
1,2,1,1,2,2,0.166
2,3,1,1,2,2,0.167
3,4,1,1,2,2,0.166
4,5,1,1,2,2,0.143
...,...,...,...,...,...,...
103,104,1,1,2,2,0.200
104,105,1,9,10,10,0.500
105,106,2,2,1,1,1.000
106,107,1,1,2,2,1.000


In [5]:
dat = pd.merge(results[["pairid", "accuracy"]], weights, on="pairid")
dat

,pairid,accuracy,v1,v2,v3,v4,weight
0,1,1.0,1,1,2,2,0.166
1,2,1.0,1,1,2,2,0.166
2,3,0.0,1,1,2,2,0.167
3,4,1.0,1,1,2,2,0.166
4,5,1.0,1,1,2,2,0.143
...,...,...,...,...,...,...,...
103,104,1.0,1,1,2,2,0.200
104,105,1.0,1,9,10,10,0.500
105,106,1.0,2,2,1,1,1.000
106,107,1.0,1,1,2,2,1.000


In [6]:
dat["score"] = dat["accuracy"] * dat["weight"]
display(dat)
print("Accuracy", sum(dat["accuracy"])/len(dat))
print("Weighted accuracy", sum(dat["score"])/sum(dat["weight"]))

,pairid,accuracy,v1,v2,v3,v4,weight,score
0,1,1.0,1,1,2,2,0.166,0.166
1,2,1.0,1,1,2,2,0.166,0.166
2,3,0.0,1,1,2,2,0.167,0.000
3,4,1.0,1,1,2,2,0.166,0.166
4,5,1.0,1,1,2,2,0.143,0.143
...,...,...,...,...,...,...,...,...
103,104,1.0,1,1,2,2,0.200,0.200
104,105,1.0,1,9,10,10,0.500,0.500
105,106,1.0,2,2,1,1,1.000,1.000
106,107,1.0,1,1,2,2,1.000,1.000


Accuracy 0.8888888888888888
Weighted accuracy 0.9215085940525001


In [7]:
import numpy as np
np.min(dat["accuracy"])

0.0